# Run Hugging Face `meta-llama/Llama-2-70b` autoregressive sampling on Inf2 & Trn1

In this example we compile and deploy the Hugging Face [meta-llama/Llama-2-70b](https://huggingface.co/meta-llama/Llama-2-70b) model for tensor parallel inference on Neuron using the `transformers-neuronx` package.

The example has the following main sections:
1. Set up the Jupyter Notebook
1. Install dependencies
1. Download the model
1. Perform autoregressive sampling using tensor parallelism

This Jupyter Notebook should be run on an Inf2 instance (`inf2.48xlarge`). To run on a larger Trn1 instance (`trn1.32xlarge`) will require changing the `tp_degree` specified in compilation section.

## Set up the Jupyter Notebook

The following steps set up Jupyter Notebook and launch this tutorial:
1. Clone the [AWS Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples) repo to your instance using
```
git clone https://github.com/aws-neuron/aws-neuron-samples.git
```
2. Navigate to the `transformers-neuronx` inference samples folder
```
cd aws-neuron-samples/torch-neuronx/transformers-neuronx/inference
```
3. Follow the instructions in [Jupyter Notebook QuickStart](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/notebook/setup-jupyter-notebook-steps-troubleshooting.html) to run Jupyter Notebook on your instance.
4. Locate this tutorial in your Jupyter Notebook session (`meta-llama-2-70b-sampling.ipynb`) and launch it. Follow the rest of the instructions in this tutorial. 

## Install Dependencies
This tutorial requires the following pip packages:

 - `torch-neuronx`
 - `neuronx-cc`
 - `sentencepiece`
 - `transformers`
 - `transformers-neuronx`


Most of these packages will be installed when configuring your environment using the [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx). The additional dependencies must be installed here:

In [ ]:
!pip install transformers-neuronx sentencepiece -U

## Download the model

Use of the Llama 2 model is governed by the Meta license and must be downloaded and converted to the standard Hugging Face format prior to running this sample. 

Follow the steps described in [meta-llama/Llama-2-70b](https://huggingface.co/meta-llama/Llama-2-70b) to get access to the Llama 2 model from Meta and download the weights and tokenizer.

After gaining access to the model checkpoints, you should be able to use the already converted checkpoints. Otherwise, if you are converting your own model, feel free to use the [conversion script](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/convert_llama_weights_to_hf.py). The script can be called with the following (example) command:
```
python src/transformers/models/llama/convert_llama_weights_to_hf.py \
    --input_dir /path/to/downloaded/llama/weights --model_size 70Bf --output_dir ./Llama-2-70b --safe_serialization
```

Note: For the purposes of this sample we assume you have saved the Llama-2-70b model in a directory called `Llama-2-70b` with the following format:
```
Llama-2-70b/
├── config.json
├── generation_config.json
├── model-00001-of-00015.safetensors
├── model-00002-of-00015.safetensors
├── model-00003-of-00015.safetensors
├── model-00004-of-00015.safetensors
├── model-00005-of-00015.safetensors
├── model-00006-of-00015.safetensors
├── model-00007-of-00015.safetensors
├── model-00008-of-00015.safetensors
├── model-00009-of-00015.safetensors
├── model-00010-of-00015.safetensors
├── model-00011-of-00015.safetensors
├── model-00012-of-00015.safetensors
├── model-00013-of-00015.safetensors
├── model-00014-of-00015.safetensors
├── model-00015-of-00015.safetensors
├── model.safetensors.index.json
├── special_tokens_map.json
├── tokenizer.json
├── tokenizer.model
└── tokenizer_config.json
```

## Perform autoregressive sampling using tensor parallelism

Now we have all of the necessary files for running `meta-llama/Llama-2-70b` autoregressive sampling. 

The memory required to host any model can be computed with:
```
total memory = bytes per parameter * number of parameters
```
When using `float16` casted weights for a 70 billion parameter model, this works out to `2 * 70B` or ~140GB of weights. Each NeuronCore has 16GB of memory which means that a 140GB model cannot fit on a single NeuronCore. In reality, the total space required is often greater than just the number of parameters due to caching attention layer projections (KV caching). This caching mechanism grows memory allocations linearly with sequence length and batch size.

To get very large language models to fit on Inf2 & Trn1, tensor parallelism is used to split weights, data, and compute across multiple NeuronCores. The number of NeuronCores that the weights are split across can be controlled by setting the `tp_degree` parameter. This parallelism degree must be chosen to ensure that the memory usage per NeuronCore will be less than the physical 16GB limit. When configuring tensor parallelism, the memory per NeuronCore can be computed with:

```
memory per core = (bytes per parameter * number of parameters) / tp_degree
```

This can be used to compute the minimum instance sizing by ensuring that the value selected for `tp_degree` results in less than 16GB allocated per NeuronCore.

Note that increasing the `tp_degree` beyond the minimum requirement almost always results in a faster model. Increasing the tensor parallelism degree improves memory bandwidth which improves model performance. To optimize performance it's recommended to use the highest tensor parallelism degree that is supported by the instance. In this sample we use tensor parallelism degree 24 to optimize performance on `inf2.48xlarge`. 

We will use the Neuron `LlamaForSampling` class to implement tensor parallelism for the Llama 2 model. The default model config supports sampling up to sequence length 2048. Tensor parallelism is enabled through the argument `tp_degree=24`. We enable `float16` casting with the `amp='f16'` flag. The model computational graph is compiled by `neuronx-cc` for optimized inference on Neuron. 

We also enable the `--enable-mixed-precision-accumulation` flag since this has been shown to improve accuracy for this model.

In [ ]:
import os
import time
import torch
from transformers import AutoTokenizer
from transformers_neuronx import LlamaForSampling

os.environ['NEURON_CC_FLAGS'] = '--enable-mixed-precision-accumulation'

# Load meta-llama/Llama-2-70b to the NeuronCores with 8-way tensor parallelism and run compilation
neuron_model = LlamaForSampling.from_pretrained(
    'Llama-2-70b',          # The reference to the safetensors checkpoint folder
    batch_size=1,           # Batch size must be determined prior to inference time.
    tp_degree=24,           # Controls the number of NeuronCores to execute on. Change to 32 for trn1.32xlarge
    amp='f16',              # This automatically casts the weights to the specified dtype.
)
neuron_model.to_neuron()

# Construct a tokenizer and encode prompt text
tokenizer = AutoTokenizer.from_pretrained('Llama-2-70b')
prompt = "Hello, I'm a language model,"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=2048, top_k=50)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq) for seq in generated_sequences]
print(f'generated sequences {generated_sequences} in {elapsed} seconds')